# Генератор PDF отчетов для всех скважин
Этот ноутбук создает высококачественные PDF визуализации для ВСЕХ скважин из merged_hourly.parquet с профессиональной обработкой NaN значений и русскими подписями.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.backends.backend_pdf as pdf_backend
import seaborn as sns
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Устанавливаем русские шрифты для matplotlib
plt.rcParams['font.family'] = 'DejaVu Sans'
try:
    import matplotlib.font_manager
    matplotlib.font_manager.fontManager.addfont('/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf')
except:
    pass  # Если шрифт не найден, используем стандартный

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configure matplotlib for better quality
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10

In [3]:
# Создаем папку для PDF файлов
pdf_output_dir = './well_pdf_plots'
os.makedirs(pdf_output_dir, exist_ok=True)
print(f"Папка для PDF файлов: {os.path.abspath(pdf_output_dir)}")

Папка для PDF файлов: /home/ruslan_safaev/alma_servie/notebooks/well_pdf_plots


In [4]:
# Load merged hourly data
data_path = '../data/processed/merged_hourly.parquet'
df = pd.read_parquet(data_path)

# Convert timestamp to datetime if it's not already
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['well_number'] = df['well_number'].astype(str)

print(f"Формат данных: {df.shape}")
print(f"Колонки: {df.columns.tolist()}")
print(f"Диапазон дат: {df['timestamp'].min()} до {df['timestamp'].max()}")
print(f"Количество уникальных скважин: {df['well_number'].nunique()}")

Формат данных: (237295, 12)
Колонки: ['timestamp', 'well_number', 'Ток фазы A', 'Выходная частота', 'Давление на приеме насоса', 'Температура двигателя', 'Устьевое давление', 'Планируемая продолжительность, мин', 'Фактическая продолжительность???, мин', 'Объемный дебит жидкости, м3/сут', 'Объемный дебит газа, м3/сут', 'Давление в коллекторе ГЗУ при замере']
Диапазон дат: 2024-01-01 00:00:00 до 2025-10-19 23:00:00
Количество уникальных скважин: 16


In [5]:
def get_well_data(df, well_number):
    """Получить данные для конкретной скважины."""
    return df[df['well_number'] == well_number].copy().sort_values('timestamp')

# Определяем группы параметров для лучшей организации
electrical_params = ['Ток фазы A', 'Выходная частота', 'Температура двигателя']
pressure_params = ['Давление на приеме насоса', 'Устьевое давление', 'Давление в коллекторе ГЗУ при замере']
production_params = ['Объемный дебит жидкости, м3/сут', 'Объемный дебит газа, м3/сут']
operational_params = ['Планируемая продолжительность, мин', 'Фактическая продолжительность???, мин']

all_params = electrical_params + pressure_params + production_params + operational_params
print(f"Всего параметров для построения: {len(all_params)}")

Всего параметров для построения: 10


In [6]:
def create_well_pdf(well_data, well_number):
    """Создать PDF для одной скважины без отображения графиков."""
    pdf_path = os.path.join(pdf_output_dir, f'скважина_{well_number}_комплексный_анализ.pdf')
    pdf_pages = pdf_backend.PdfPages(pdf_path)
    
    # Создаем график для каждого параметра на отдельной странице
    for param in all_params:
        # Создаем фигуру для этого параметра
        fig, ax = plt.subplots(figsize=(12, 8))
        fig.suptitle(f'Скважина {well_number} - {param}', fontsize=16, fontweight='bold')
        
        # Разделяем валидные и NaN данные
        valid_data = well_data[well_data[param].notna()]
        nan_data = well_data[well_data[param].isna()]
        
        # Строим валидные точки данных
        if len(valid_data) > 0:
            ax.plot(valid_data['timestamp'], valid_data[param], 
                   linewidth=2.0, alpha=0.8, label='Валидные данные', 
                   color='#1f77b4', marker='o', markersize=3, markevery=max(1, len(valid_data)//50))
        
        # Отмечаем периоды NaN профессиональными градиентными затемненными областями
        if len(nan_data) > 0:
            # Группируем последовательные периоды NaN
            nan_periods = []
            if len(nan_data) > 0:
                current_start = nan_data.iloc[0]['timestamp']
                prev_timestamp = current_start
                
                for _, row in nan_data.iterrows():
                    if row['timestamp'] - prev_timestamp > pd.Timedelta(hours=2):
                        nan_periods.append((current_start, prev_timestamp))
                        current_start = row['timestamp']
                    prev_timestamp = row['timestamp']
                nan_periods.append((current_start, prev_timestamp))
                
                # Применяем градиентное затемнение для каждого периода NaN
                for start, end in nan_periods:
                    # Создаем градиентный эффект для отсутствующих данных
                    ax.axvspan(start, end, alpha=0.15, color='#ff6b6b', 
                              edgecolor='#c92a2a', linewidth=0.5, linestyle='--')
                    
                    # Добавляем тонкий узор для лучшей видимости
                    ax.axvspan(start, end, alpha=0.05, color='#495057', 
                              hatch='///', linewidth=0)
        
        # Форматируем подграфик
        ax.set_title(f'{param}', fontweight='bold', fontsize=14, pad=20)
        ax.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
        ax.tick_params(axis='x', rotation=45, labelsize=10)
        ax.tick_params(axis='y', labelsize=10)
        
        # Форматируем даты на оси X с оптимальным интервалом для лучшей читаемости
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
        
        # Определяем оптимальный интервал для отметок на оси X в зависимости от диапазона дат
        date_range = well_data['timestamp'].max() - well_data['timestamp'].min()
        if date_range.days <= 7:
            # Для недели или менее - показываем каждый день
            ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
        elif date_range.days <= 30:
            # Для месяца - показываем каждые 3 дня
            ax.xaxis.set_major_locator(mdates.DayLocator(interval=3))
        elif date_range.days <= 90:
            # Для квартала - показываем каждую неделю
            ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=1))
        else:
            # Для больших периодов - показываем каждые 2 недели
            ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=2))
        
        # Добавляем вспомогательные отметки для лучшей ориентации
        ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
        
        # Добавляем профессиональный текстовый блок со статистикой
        if len(valid_data) > 0:
            stats_text = f'Статистика:\n'
            stats_text += f'Среднее: {valid_data[param].mean():.2f}\n'
            stats_text += f'Стд. откл.: {valid_data[param].std():.2f}\n'
            stats_text += f'Минимум: {valid_data[param].min():.2f}\n'
            stats_text += f'Максимум: {valid_data[param].max():.2f}\n'
            stats_text += f'Пропуски: {len(nan_data)}/{len(well_data)} ({len(nan_data)/len(well_data)*100:.1f}%)'
            
            ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, 
                   verticalalignment='top', 
                   bbox=dict(boxstyle='round,pad=0.5', facecolor='white', 
                            edgecolor='#dee2e6', alpha=0.95, linewidth=1),
                   fontsize=10, family='monospace')
        
        # Добавляем легенду
        if len(valid_data) > 0 or len(nan_data) > 0:
            legend_elements = []
            if len(valid_data) > 0:
                legend_elements.append(plt.Line2D([0], [0], color='#1f77b4', lw=2, label='Валидные данные'))
            if len(nan_data) > 0:
                legend_elements.append(plt.Rectangle((0, 0), 1, 1, fc='#ff6b6b', alpha=0.15, 
                                                   edgecolor='#c92a2a', label='Отсутствующие данные'))
            ax.legend(handles=legend_elements, loc='upper right', fontsize=10, 
                     framealpha=0.95, fancybox=True, shadow=True)
        
        # Добавляем информацию о периоде
        period_text = f'Период: {well_data["timestamp"].min().strftime("%Y-%m-%d")} до {well_data["timestamp"].max().strftime("%Y-%m-%d")}'
        fig.text(0.5, 0.02, period_text, ha='center', fontsize=11, 
                bbox=dict(boxstyle='round,pad=0.3', facecolor='#f8f9fa', 
                         edgecolor='#dee2e6', alpha=0.8))
        
        plt.tight_layout(rect=[0, 0.05, 1, 0.96])
        
        # Сохраняем в PDF без отображения
        pdf_pages.savefig(fig, dpi=300, bbox_inches='tight')
        
        # Закрываем фигуру без отображения
        plt.close(fig)
    
    pdf_pages.close()
    return pdf_path

In [7]:
def generate_all_wells_pdfs(df):
    """Генерирует PDF файлы для ВСЕХ скважин в наборе данных без ограничений."""
    all_wells = df['well_number'].unique()
    total_wells = len(all_wells)
    
    print(f"Начинаем генерацию PDF для ВСЕХ {total_wells} скважин...")
    print(f"Ожидаемое время выполнения: ~{total_wells * 2} секунд")
    
    successful_generations = 0
    failed_wells = []
    pdf_files = []
    
    start_time = datetime.now()
    
    for i, well_number in enumerate(all_wells, 1):
        try:
            elapsed_time = (datetime.now() - start_time).total_seconds()
            remaining_time = elapsed_time / i * (total_wells - i)
            
            print(f"\rОбработка скважины {well_number} ({i}/{total_wells}) - Элапс: {elapsed_time:.0f}с, Осталось: {remaining_time:.0f}с", end="")
            
            well_data = get_well_data(df, well_number)
            
            if len(well_data) > 0:
                pdf_path = create_well_pdf(well_data, well_number)
                pdf_files.append(pdf_path)
                successful_generations += 1
            else:
                print(f"\nПредупреждение: Нет данных для скважины {well_number}")
                failed_wells.append(well_number)
                
        except Exception as e:
            print(f"\nОшибка при обработке скважины {well_number}: {str(e)}")
            failed_wells.append(well_number)
    
    total_time = (datetime.now() - start_time).total_seconds()
    print(f"\n\nГенерация завершена за {total_time:.1f} секунд!")
    print(f"Успешно создано PDF: {successful_generations}/{total_wells}")
    
    if failed_wells:
        print(f"Скважины с ошибками: {failed_wells}")
    
    # Создаем сводный отчет
    create_summary_report(df, successful_generations, total_wells, failed_wells)
    
    return successful_generations, failed_wells, pdf_files

In [8]:
def create_summary_report(df, successful_count, total_count, failed_wells):
    """Создает сводный отчет о качестве данных."""
    # Вычисляем метрики качества данных
    quality_metrics = []
    for param in all_params:
        if param in df.columns:
            total_count_param = len(df)
            valid_count = df[param].notna().sum()
            missing_count = total_count_param - valid_count
            missing_percentage = (missing_count / total_count_param) * 100
            
            quality_metrics.append({
                'Параметр': param,
                'Всего записей': total_count_param,
                'Валидных записей': valid_count,
                'Пропущенных записей': missing_count,
                'Пропущено %': missing_percentage
            })
    
    quality_df = pd.DataFrame(quality_metrics)
    
    # Создаем визуализацию для сводного отчета
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
    
    # График процентов пропущенных данных
    colors = ['red' if x > 50 else 'orange' if x > 20 else 'green' for x in quality_df['Пропущено %']]
    bars = ax1.barh(range(len(quality_df)), quality_df['Пропущено %'], color=colors, alpha=0.7)
    ax1.set_yticks(range(len(quality_df)))
    ax1.set_yticklabels([p[:35] + '...' if len(p) > 35 else p for p in quality_df['Параметр']])
    ax1.set_xlabel('Процент пропущенных данных (%)')
    ax1.set_title('Обзор качества данных', fontweight='bold', fontsize=14)
    ax1.grid(True, alpha=0.3)
    
    # Добавляем метки процентов на столбцах
    for i, (bar, pct) in enumerate(zip(bars, quality_df['Пропущено %'])):
        ax1.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, 
                f'{pct:.1f}%', va='center', fontsize=9)
    
    # Круговая диаграмма доступности данных
    total_valid = quality_df['Валидных записей'].sum()
    total_missing = quality_df['Пропущенных записей'].sum()
    
    wedges, texts, autotexts = ax2.pie([total_valid, total_missing], 
           labels=['Валидные данные', 'Пропущенные данные'], 
           colors=['green', 'red'], 
           autopct='%1.1f%%',
           startangle=90,
           textprops={'fontsize': 12})
    ax2.set_title('Общая доступность данных', fontweight='bold', fontsize=14)
    
    plt.suptitle(f'Сводный отчет по качеству данных\nУспешно обработано скважин: {successful_count}/{total_count}', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    # Сохраняем сводный отчет в PDF
    report_path = os.path.join(pdf_output_dir, 'сводный_отчет_качества_данных.pdf')
    pdf_pages = pdf_backend.PdfPages(report_path)
    pdf_pages.savefig(fig, dpi=300, bbox_inches='tight')
    pdf_pages.close()
    plt.close(fig)
    
    print(f"Сводный отчет сохранен как '{report_path}'")
    return quality_df

In [9]:
def list_generated_pdfs():
    """Показывает список всех сгенерированных PDF файлов."""
    pdf_files = [f for f in os.listdir(pdf_output_dir) if f.endswith('.pdf')]
    
    print(f"\nСгенерированные PDF файлы в папке '{pdf_output_dir}':")
    print("="*80)
    
    if pdf_files:
        total_size = 0
        for i, file in enumerate(pdf_files, 1):
            file_path = os.path.join(pdf_output_dir, file)
            file_size = os.path.getsize(file_path) / (1024 * 1024)  # Размер в МБ
            mod_time = datetime.fromtimestamp(os.path.getmtime(file_path))
            total_size += file_size
            print(f"{i:2d}. {file:<45} | {file_size:6.2f} МБ | {mod_time.strftime('%Y-%m-%d %H:%M:%S')}")
        print("="*80)
        print(f"Всего файлов: {len(pdf_files)} | Общий размер: {total_size:.2f} МБ")
    else:
        print("PDF файлы еще не сгенерированы.")
    
    return pdf_files

In [10]:
# ЗАПУСК ГЕНЕРАЦИИ PDF ДЛЯ ВСЕХ СКВАЖИН
# Это может занять несколько минут в зависимости от количества скважин
print("Начинаем массовую генерацию PDF отчетов...")
successful, failed, pdf_files = generate_all_wells_pdfs(df)
print(f"\nГенерация завершена! Успешно: {successful}, Ошибок: {len(failed)}")

Начинаем массовую генерацию PDF отчетов...
Начинаем генерацию PDF для ВСЕХ 16 скважин...
Ожидаемое время выполнения: ~32 секунд
Обработка скважины 978 (16/16) - Элапс: 137с, Осталось: 0сс5с

Генерация завершена за 146.2 секунд!
Успешно создано PDF: 16/16
Сводный отчет сохранен как './well_pdf_plots/сводный_отчет_качества_данных.pdf'

Генерация завершена! Успешно: 16, Ошибок: 0


In [11]:
# Показываем список всех сгенерированных файлов
list_generated_pdfs()


Сгенерированные PDF файлы в папке './well_pdf_plots':
 1. скважина_40617_комплексный_анализ.pdf         |   0.16 МБ | 2025-10-25 19:06:01
 2. сводный_отчет_качества_данных.pdf             |   0.04 МБ | 2025-10-25 19:07:32
 3. скважина_50069_комплексный_анализ.pdf         |   0.32 МБ | 2025-10-25 19:06:54
 4. скважина_962_комплексный_анализ.pdf           |   0.27 МБ | 2025-10-25 19:07:23
 5. скважина_1018_комплексный_анализ.pdf          |   0.49 МБ | 2025-10-25 19:05:18
 6. скважина_1756_комплексный_анализ.pdf          |   0.21 МБ | 2025-10-25 19:05:28
 7. скважина_454_комплексный_анализ.pdf           |   0.27 МБ | 2025-10-25 19:06:28
 8. скважина_40833_комплексный_анализ.pdf         |   0.35 МБ | 2025-10-25 19:06:18
 9. скважина_1963_комплексный_анализ.pdf          |   0.22 МБ | 2025-10-25 19:05:36
10. скважина_737_комплексный_анализ.pdf           |   0.35 МБ | 2025-10-25 19:07:13
11. скважина_978_комплексный_анализ.pdf           |   0.29 МБ | 2025-10-25 19:07:32
12. скважина_48359_ко

['скважина_40617_комплексный_анализ.pdf',
 'сводный_отчет_качества_данных.pdf',
 'скважина_50069_комплексный_анализ.pdf',
 'скважина_962_комплексный_анализ.pdf',
 'скважина_1018_комплексный_анализ.pdf',
 'скважина_1756_комплексный_анализ.pdf',
 'скважина_454_комплексный_анализ.pdf',
 'скважина_40833_комплексный_анализ.pdf',
 'скважина_1963_комплексный_анализ.pdf',
 'скважина_737_комплексный_анализ.pdf',
 'скважина_978_комплексный_анализ.pdf',
 'скважина_48359_комплексный_анализ.pdf',
 'скважина_50184_комплексный_анализ.pdf',
 'скважина_339_комплексный_анализ.pdf',
 'скважина_48361_комплексный_анализ.pdf',
 'скважина_40620_комплексный_анализ.pdf',
 'скважина_234_комплексный_анализ.pdf']

## Инструкции по использованию

### Что делает этот ноутбук:

1. **Автоматически генерирует PDF** для ВСЕХ скважин в наборе данных
2. **Не отображает графики** в интерфейсе Jupyter (только сохраняет в PDF)
3. **Создает профессиональные отчеты** с русскими подписями
4. **Показывает прогресс** и время выполнения

### Результаты:

- Папка `well_pdf_plots/` содержит PDF файлы для каждой скважины
- `скважина_{номер}_комплексный_анализ.pdf` - детальный анализ каждой скважины
- `сводный_отчет_качества_данных.pdf` - общий отчет по всему набору данных

### Формат каждого PDF:

- 12 страниц (по одной на каждый параметр)
- Высокое качество 300 DPI
- Профессиональная визуализация NaN значений
- Полная статистика по каждому параметру
- Русские подписи

### Оптимизация оси времени:

Настройка интервалов оси X автоматически адаптируется под диапазон дат:
- **≤ 7 дней**: отметки каждый день
- **≤ 30 дней**: отметки каждые 3 дня
- **≤ 90 дней**: отметки каждую неделю
- **> 90 дней**: отметки каждые 2 недели

### Цветовая схема:

- **Синий линии**: Валидные данные
- **Красные градиентные области**: Периоды отсутствия данных
- **Серые узоры**: Усиление визуализации пропусков

### Профессиональная обработка NaN:

- **Градиентное затемнение**: Полупрозрачные красные области указывают на отсутствующие данные
- **Узорное покрытие**: Тонкие диагональные линии для лучшей видимости
- **Группировка периодов**: Последовательные NaN значения группируются в непрерывные блоки
- **Профессиональные рамки**: Пунктирные границы четко определяют области отсутствующих данных

### Запуск:

Просто выполните ячейки последовательно. Последняя ячейка начнет массовую генерацию всех PDF файлов.

### Все файлы сохраняются в папку `well_pdf_plots/` с русскими названиями и профессиональным оформлением.